
# Parameter-Efficient Fine-Tuning (PEFT) 실습
**작성일**: 2025-07-18 by Yeongtak Oh (Credit : DSAIL LAb, SNU)

**목표**:  
- 사전 학습된 언어 모델에 대해 효율적인 파인튜닝 방법인 **LoRA (Low-Rank Adaptation)** 이해  
- HuggingFace `peft` 라이브러리를 사용한 실습  
- Colab에서 실행 가능한 코드 제공  
- 예제 데이터셋: SST-2 (감정 분류)

---

> 본 노트북은 HD현대 실습을 위해 교육용 자료로서 준비되었으며, PEFT의 핵심 개념 및 실습을 포함합니다.  
> **참고:** 이 노트북은 HuggingFace `transformers`, `datasets`, `peft` 라이브러리 사용을 기반으로 합니다.


In [ ]:

!pip install transformers datasets peft accelerate bitsandbytes --quiet


In [ ]:
!pip install -U datasets transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)


In [ ]:
# ✳️ 기본 모델 및 토크나이저 로드
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)


In [ ]:
from datasets import load_dataset
# ✳️ SST2 데이터셋 로드 및 전처리
dataset = load_dataset("stanfordnlp/sst2")

def tokenize_fn(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=128)

encoded = dataset.map(tokenize_fn, batched=True)
encoded = encoded.remove_columns(["sentence", "idx"])
encoded.set_format("torch")
print(encoded)

In [ ]:
import re
sentences = list(dataset['train']['sentence'])

In [ ]:
" ".join([s.strip() for s in sentences])

In [ ]:
from peft import LoraConfig

# ✳️ LoRA 설정 및 모델 래핑
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]  # DistilBERT용. 또는 ["query", "value"] 등
)

lora_model = get_peft_model(base_model, peft_config)
lora_model.print_trainable_parameters()


In [ ]:
peft_config

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# ✳️ 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./lora_sst2",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=encoded["train"].shuffle(seed=42).select(range(1000)),  # 샘플 1000개 사용
    eval_dataset=encoded["validation"].select(range(300)),
)

trainer.train()


In [ ]:
training_args

In [ ]:
from matplotlib import pyplot as plt

plt.title('loss_plot')
plt.xlabel('num of logging steps')
plt.ylabel('training loss')
plt.plot([trainer.state.log_history[i]['loss'] for i in range(len(trainer.state.log_history)-1)], label='LoRA loss')
plt.legend()

In [ ]:
# ✳️ 평가 및 저장
metrics = trainer.evaluate()
print("\n📊 평가 결과:", metrics)

# LoRA adapter만 저장
lora_model.save_pretrained("./lora_adapter_sst2")
print("💾 LoRA adapter 저장 완료: ./lora_adapter_sst2")



---

## 📌 LoRA란?

LoRA는 **Low-Rank Adaptation of Large Language Models**의 줄임말로,  
사전학습된 모델의 파라미터는 **동결(freeze)** 하고,  
일부 층에 대해 **low-rank 행렬을 추가 학습**함으로써  
**매우 적은 수의 학습 가능한 파라미터**로도 fine-tuning이 가능하게 하는 기법입니다.

> 📎 참고 문헌: [https://arxiv.org/abs/2106.09685](https://arxiv.org/abs/2106.09685)

---

## 📌 실습 핵심 포인트

- `AutoModelForSequenceClassification`: 분류용 pre-trained 모델 사용  
- `peft.LoraConfig`: 파인튜닝할 레이어 설정 (dropout, rank 등)  
- `get_peft_model`: 기존 모델을 LoRA 적용 가능한 형태로 래핑  
- **trainable parameters**만 출력 확인 → 기존 모델 대비 약 0.1~1% 수준  
- SST-2 감정 분류 데이터셋 사용 (긍정/부정)

---

## 📊 결과 확인 예시

- 정확도(accuracy), 손실(loss) 등 기본 평가 지표 출력  
- `save_pretrained()` 호출 시 full model이 아니라 LoRA adapter만 저장됨  

---  



## 📝 퀴즈 및 복습 질문

1. LoRA의 핵심 아이디어는 무엇인가요? 어떤 장점이 있나요?  
2. LoRA를 적용하기 위해 어떤 구성요소들이 필요한가요?  
3. Full fine-tuning과 비교해 PEFT의 제한사항은 무엇이 있을까요?

> 아래 셀에 자유롭게 작성해 보세요.


In [ ]:
# 여기에 작성하세요